In [1]:
import pandas as pd
import numpy as np

from talkingdata import run
from models import XGBEstimator, PhoneBrandEstimator

%load_ext autoreload
%autoreload 2

In [2]:
gatrain = pd.read_csv('data/gender_age_train.csv')
pbrand = pd.read_csv('data/phone_brand_device_model.csv')

In [3]:
gatest = pd.read_csv('data/gender_age_test.csv')

In [4]:
pbrand.phone_brand.describe()

count     187245
unique       131
top           小米
freq       43210
Name: phone_brand, dtype: object

In [5]:
gaptrain = gatrain.merge(pbrand.drop_duplicates('device_id',keep='first'), on='device_id')

In [6]:
gaptrain.columns

Index(['device_id', 'gender', 'age', 'group', 'phone_brand', 'device_model'], dtype='object')

In [7]:
translations = pd.read_csv('data/phone_brand_chinese_to_english.csv', sep=',')

In [8]:
translations.head()

,ch,eng
0,三星,samsung
1,天语,Ktouch
2,海信,hisense
3,联想,lenovo
4,欧比,obi


In [9]:
translations.ch.describe()

count       55
unique      55
top       摩托罗拉
freq         1
Name: ch, dtype: object

Como podemos ver, hay muchos menos modelos con traducciones (55) que el total (131)

In [10]:
for chb, engb in zip(translations['ch'], translations['eng']):
    gaptrain.replace(to_replace=chb, value=engb, inplace=True)

#gaptrain.replace(to_replace=translations['ch'], value=translations['eng'], inplace=True)

Acabo de descubrir que 小米 es la traduccion de Xiaomi. Y 红米 es Red Rice

In [11]:
gaptrain.replace(to_replace=['小米', '红米'], value=['Xiaomi', 'Red Rice'], inplace=True)

In [12]:
gaptrain.device_id = gaptrain.device_id.astype('category')
gaptrain.device_model = gaptrain.device_model.astype('category')
gaptrain.phone_brand = gaptrain.phone_brand.astype('category')

In [13]:
gaptest = gatest.merge(pbrand.drop_duplicates('device_id',keep='first'), on='device_id')

for chb, engb in zip(translations['ch'], translations['eng']):
    gaptest.replace(to_replace=chb, value=engb, inplace=True)
gaptest.replace(to_replace=['小米', '红米'], value=['Xiaomi', 'Red Rice'], inplace=True)

In [14]:
dataset = gaptrain.drop(['gender', 'age'], axis=1)

In [15]:
dataset.head(10)

,device_id,group,phone_brand,device_model
0,-8076087639492063270,M32-38,Xiaomi,MI 2
1,-2897161552818060146,M32-38,Xiaomi,MI 2
2,-8260683887967679142,M32-38,Xiaomi,MI 2
3,-4938849341048082022,M29-31,Xiaomi,红米note
4,245133531816851882,M29-31,Xiaomi,MI 3
5,-1297074871525174196,F24-26,OPPO,R1C
6,236877999787307864,M32-38,酷派,F2
7,-8098239495777311881,M32-38,Xiaomi,红米note
8,176515041953473526,M32-38,vivo,X3T
9,1596610250680140042,F33-42,samsung,Galaxy S4


Voy a agregar apps

In [16]:
#appsevents = pd.read_csv('data/app_events.csv', usecols=['event_id','app_id','is_active'], dtype={'is_active':bool})
appsevents = pd.read_csv('data/app_events.csv', usecols=['event_id','app_id'])
appsevents.head()

,event_id,app_id
0,2,5927333115845830913
1,2,-5720078949152207372
2,2,-1633887856876571208
3,2,-653184325010919369
4,2,8693964245073640147


In [17]:
#events = pd.read_csv('data/events.csv', parse_dates=['timestamp'], usecols=['event_id', 'device_id', 'timestamp'])
events = pd.read_csv('data/events.csv', usecols=['event_id', 'device_id'])
events.head()

,event_id,device_id
0,1,29182687948017175
1,2,-6401643145415154744
2,3,-4833982096941402721
3,4,-6815121365017318426
4,5,-5373797595892518570


In [18]:
appsevents = appsevents.merge(events, on='event_id')
appsevents = appsevents.drop('event_id', axis=1)
appsevents = appsevents.drop_duplicates()
appsevents.head()

,app_id,device_id
0,5927333115845830913,-6401643145415154744
1,-5720078949152207372,-6401643145415154744
2,-1633887856876571208,-6401643145415154744
3,-653184325010919369,-6401643145415154744
4,8693964245073640147,-6401643145415154744


Mergear dentro del dataset y borrar features que no voy a utilizar.

In [19]:
dataset = dataset.merge(appsevents, on='device_id')
dataset.head()

,device_id,group,phone_brand,device_model,app_id
0,-8260683887967679142,M32-38,Xiaomi,MI 2,4287147352639325907
1,-8260683887967679142,M32-38,Xiaomi,MI 2,-4085686983594738208
2,-8260683887967679142,M32-38,Xiaomi,MI 2,6324194957022958681
3,-8260683887967679142,M32-38,Xiaomi,MI 2,-8687507491028761317
4,-8260683887967679142,M32-38,Xiaomi,MI 2,877288515501232570


In [20]:
gaptest = gaptest.merge(appsevents, on='device_id')
gaptest.head()

,device_id,phone_brand,device_model,app_id
0,1002079943728939269,Xiaomi,小米note,5927333115845830913
1,1002079943728939269,Xiaomi,小米note,-6590029937880196169
2,1002079943728939269,Xiaomi,小米note,-1488214531155612478
3,1002079943728939269,Xiaomi,小米note,-9142957261685295367
4,1002079943728939269,Xiaomi,小米note,8693964245073640147


In [21]:
# Leak
dataset['row'] = dataset.index
gaptest['row'] = gaptest.index

In [22]:
dataset.columns

Index(['device_id', 'group', 'phone_brand', 'device_model', 'app_id', 'row'], dtype='object')

In [23]:
gaptest.columns

Index(['device_id', 'phone_brand', 'device_model', 'app_id', 'row'], dtype='object')

In [20]:
# Logistic without apps
results, rpred = run(PhoneBrandAppsEstimator, dataset.dropna(), valid=gaptest)
pred = pd.DataFrame(rpred[:len(rpred)/4], columns=results.columns[2:])
pred.insert(0, 'device_id', gatest.device_id)
pred.to_csv('data/valid_predictions_lrlk.csv', index=False)

Predicted LogLoss: 0_ 2.40066
Predicted LogLoss: 1_ 2.40196
Predicted LogLoss: 2_ 2.40330
Predicted LogLoss: 3_ 2.39789


/home/gonzalo/virtualenvs/machine-learning/lib/python3.4/site-packages/ipykernel/__main__.py:2: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  from ipykernel import kernelapp as app


In [38]:
# XGBoost without apps
results, rpred = run(XGBEstimator, dataset.dropna(), valid=gaptest)
pred = pd.DataFrame(rpred[:len(rpred)/4], columns=results.columns[2:])
pred.insert(0, 'device_id', gatest.device_id)
pred.to_csv('data/valid_predictions_xgb.csv', index=False)

LogLoss it 0: 2.87337
LogLoss it 1: 3.30028
LogLoss it 2: 3.09062
LogLoss it 3: 3.06112


/home/gonzalo/virtualenvs/machine-learning/lib/python3.4/site-packages/ipykernel/__main__.py:2: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  from ipykernel import kernelapp as app


CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 7.87 µs


In [24]:
#XGBoost con apps ids
results, rpred = run(XGBEstimator, dataset.dropna(), valid=gaptest)
pred = pd.DataFrame(rpred[:len(rpred)/4], columns=results.columns[2:])
pred.insert(0, 'device_id', gatest.device_id)
pred.to_csv('data/valid_predictions_xgb.csv', index=False)

LogLoss it 0: 6.13999
LogLoss it 1: 5.21842
LogLoss it 2: 6.17781
LogLoss it 3: 6.48236


/home/gonzalo/virtualenvs/machine-learning/lib/python3.4/site-packages/ipykernel/__main__.py:3: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  app.launch_new_instance()


In [110]:
#Logistic regr con apps ids
results, rpred = run(PhoneBrandEstimator, dataset.dropna(), valid=gaptest)
pred = pd.DataFrame(rpred[:len(rpred)/4], columns=results.columns[2:])
pred.insert(0, 'device_id', gatest.device_id)
pred.to_csv('data/valid_predictions_xgb.csv', index=False)

LogLoss it n 0: 2.22377
LogLoss it n 1: 2.21093
LogLoss it n 2: 2.20811
LogLoss it n 3: 2.20655


ValueError: all the input arrays must have same number of dimensions

In [26]:
pred.head()

,device_id,F23-,F24-26,F27-28,F29-32,F33-42,F43+,M22-,M23-26,M27-28,M29-31,M32-38,M39+
0,1.002080e+18,0.029786,0.052047,0.049851,0.066659,0.054519,0.032909,0.065532,0.132401,0.120194,0.108487,0.186165,0.101452
1,-1.547860e+18,0.029786,0.052047,0.049851,0.066659,0.054519,0.032909,0.065532,0.132401,0.120194,0.108487,0.186165,0.101452
2,7.374582e+18,0.029786,0.052047,0.049851,0.066659,0.054519,0.032909,0.065532,0.132401,0.120194,0.108487,0.186165,0.101452
3,-6.220210e+18,0.029786,0.052047,0.049851,0.066659,0.054519,0.032909,0.065532,0.132401,0.120194,0.108487,0.186165,0.101452
4,-5.893464e+18,0.029786,0.052047,0.049851,0.066659,0.054519,0.032909,0.065532,0.132401,0.120194,0.108487,0.186165,0.101452


In [29]:
pred['device_id'] = pred.device_id.astype('category')

In [30]:
pred.device_id.describe()

count     1.120710e+05
unique    1.120700e+05
top       1.186608e+18
freq      2.000000e+00
Name: device_id, dtype: float64

In [43]:
len(gatest)

112071

In [44]:
gatest.device_id.describe()

count    1.120710e+05
mean    -2.367461e+16
std      5.331855e+18
min     -9.223322e+18
25%     -4.661036e+18
50%     -3.107321e+16
75%      4.581985e+18
max      9.223069e+18
Name: device_id, dtype: float64

In [112]:
#Logistic regr con apps ids
results, rpred = run(PhoneBrandEstimator, dataset.dropna(), valid=gaptest)
pred = pd.DataFrame(rpred, columns=results.columns[2:])
pred.insert(0, 'device_id', gatest.device_id)

LogLoss it 0: 2.22377
LogLoss it 1: 2.21093
LogLoss it 2: 2.20811
LogLoss it 3: 2.20655


In [114]:
pred.tail()

,device_id,F23-,F24-26,F27-28,F29-32,F33-42,F43+,M22-,M23-26,M27-28,M29-31,M32-38,M39+
1387332,NaN,0.025589,0.043393,0.030795,0.045605,0.085327,0.052979,0.061811,0.076307,0.067286,0.142609,0.186254,0.182046
1387333,NaN,0.025589,0.043393,0.030795,0.045605,0.085327,0.052979,0.061811,0.076307,0.067286,0.142609,0.186254,0.182046
1387334,NaN,0.025589,0.043393,0.030795,0.045605,0.085327,0.052979,0.061811,0.076307,0.067286,0.142609,0.186254,0.182046
1387335,NaN,0.025589,0.043393,0.030795,0.045605,0.085327,0.052979,0.061811,0.076307,0.067286,0.142609,0.186254,0.182046
1387336,NaN,0.025589,0.043393,0.030795,0.045605,0.085327,0.052979,0.061811,0.076307,0.067286,0.142609,0.186254,0.182046


In [118]:
len(pred.drop_duplicates(subset='device_id').dropna())

112070

In [119]:
len(gatest.drop_duplicates(subset='device_id').dropna())

112071

In [124]:
pred.drop_duplicates(subset='device_id').dropna().tail()

,device_id,F23-,F24-26,F27-28,F29-32,F33-42,F43+,M22-,M23-26,M27-28,M29-31,M32-38,M39+
112066,4.280901e+18,0.086622,0.075646,0.000685,0.018894,0.049052,0.024118,0.215555,0.197651,0.045551,0.111104,0.065173,0.10995
112067,8.185348e+17,0.086622,0.075646,0.000685,0.018894,0.049052,0.024118,0.215555,0.197651,0.045551,0.111104,0.065173,0.10995
112068,-8.956851e+18,0.086622,0.075646,0.000685,0.018894,0.049052,0.024118,0.215555,0.197651,0.045551,0.111104,0.065173,0.10995
112069,6.097318e+18,0.086622,0.075646,0.000685,0.018894,0.049052,0.024118,0.215555,0.197651,0.045551,0.111104,0.065173,0.10995
112070,6.224212e+17,0.086622,0.075646,0.000685,0.018894,0.049052,0.024118,0.215555,0.197651,0.045551,0.111104,0.065173,0.10995


In [100]:
gatest.device_id.describe()

count    1.120710e+05
mean    -2.367461e+16
std      5.331855e+18
min     -9.223322e+18
25%     -4.661036e+18
50%     -3.107321e+16
75%      4.581985e+18
max      9.223069e+18
Name: device_id, dtype: float64

In [129]:
len(results.drop_duplicates(subset='device_id'))/4

5822.5